This notebook aims to compute the technical lag and opportunity lag of workflows w.r.t. actions. To do so, for each step relying on an Action and each snapshot, we identify which release of the target Action is selected, what was the first missed release (i.e., a more recent release not selected) and what was the last missed release (i.e., the latest release not selected). Then, the technical lag is computed as the difference (in time and number of versions) between the selected release and the last missed one (hence representing how far in the past you are lagging behind); and the opportunity lag is computed as the difference between the snapshot date and the first release that was missed (hence representing for how long you could have updated your step). 

The strategy we follow to detect which release is selected is (1) find a release whose version number is exactly the one that is used in the step; (2) find the latest release for which the version number used in the step is a prefix (e.g., take "2.3.4" if the step is using "2.3" or "2").

In [1]:
import pandas as pd
import numpy as np
import multiprocessing

from tqdm import tqdm
from functools import partial

## Load data

In [2]:
df_steps = pd.merge(
    left=(
        # Load snapshots
        pd.read_csv('../data/workflow_files.csv.gz')
        .assign(snapshot=lambda d: pd.to_datetime(d.date))
        [['snapshot', 'repository', 'workflow']]
    ),
    right=(
        # Load workflow steps
        pd.read_csv('../data/steps.csv.gz')
        # Filter steps relying on a publicly available Action
        [lambda d: ~d.uses.isnull()]
        [lambda d: ~d.uses.str.startswith(('docker://', './', 'http'))]
        [lambda d: d.uses.str.count('/') == 1]
        # Extract Action name and the anchor used to refer to it
        .assign(action=lambda d: d.uses.str.split('@', n=1).str[0].str.lower())
        .assign(anchor=lambda d: d.uses.str.split('@', n=1).str[1])
        [['workflow', 'job', 'pos', 'action', 'anchor']]
    ),
    how='inner', 
    on=['workflow']
)

In [3]:
df_releases_unsorted = (
    pd.read_csv('../data/releases.csv.gz')
    .assign(date=lambda d: pd.to_datetime(d.date).dt.tz_convert(None))
    .assign(action=lambda d: (d.owner + '/' + d.repo).str.lower())
    [['action', 'release', 'date']]
)

In [4]:
display(df_steps.head())
display(df_releases_unsorted.head())

,snapshot,repository,workflow,job,pos,action,anchor
0,2022-09-01,davidb/scala-maven-plugin,0cfbb13493b2fb0457dbe5ccdcad07b6cf05b435495068...,build,1,actions/checkout,v3
1,2022-09-01,davidb/scala-maven-plugin,0cfbb13493b2fb0457dbe5ccdcad07b6cf05b435495068...,build,2,actions/setup-java,v3
2,2022-09-01,davidb/scala-maven-plugin,0cfbb13493b2fb0457dbe5ccdcad07b6cf05b435495068...,build,3,actions/cache,v3
3,2022-08-01,davidb/scala-maven-plugin,0cfbb13493b2fb0457dbe5ccdcad07b6cf05b435495068...,build,1,actions/checkout,v3
4,2022-08-01,davidb/scala-maven-plugin,0cfbb13493b2fb0457dbe5ccdcad07b6cf05b435495068...,build,2,actions/setup-java,v3


,action,release,date
0,actions/checkout,v3.0.2,2022-04-21 14:56:58
1,actions/checkout,v2.4.2,2022-04-21 16:04:02
2,actions/checkout,v3.0.1,2022-04-14 18:22:54
3,actions/checkout,v2.4.1,2022-04-14 16:14:18
4,actions/checkout,v3.0.0,2022-03-01 17:48:27


## Release order

Some actions maintain multiple branches in parallel. For example, the `actions/checkout` Action maintains the v2 and v3 branches in parallel.
As a consequence, sorting releases by date might lead to an inaccurate technical or opportunity lag. Consider for example a step referring to `actions/checkout` using the `v3` tag. This tag selects version `v3.0.2` released on `2022-04-21 14:56:58`. If we sort releases by date, then version `v2.4.2` will be considered as missed (hence `v3` does not target the latest release) because it was released on `2022-04-21 16:04:02` (around one hour later than `v3.0.2`). 

To avoid this situation, especially in the case of widely used Actions such as `actions/checkout`, we detect whether an Action uses a component-based version number for its releases and, if so, we sort the releases based on these components instead of their release dates. 

In [10]:
df_releases = (
    df_releases_unsorted
    # Get component-based version's components
    .pipe(lambda df:
        df.join(
            df.release.str.extract(r'^(?:(?:v|V)\.?)?(?P<major>\d+)(?:\.(?P<minor>\d+)(?:\.(?P<patch>\d+))?)?(?P<misc>.*)$')
            # Because "v1" should be before "v1.2.3"
            .fillna({'minor': 0, 'patch': 0})
            # Because we want to compare version numbers based on the "number" not on "text"
            .assign(major=lambda d: d.major.astype(int, errors='ignore'))
            .assign(minor=lambda d: d.minor.astype(int))
            .assign(patch=lambda d: d.patch.astype(int))
        )
    )
    .groupby('action', sort=False, as_index=False)
    # Sort releases by version if they are component-based, by date otherwise
    .apply(lambda g: 
        g
        .sort_values('date' if g.major.isnull().max() else ['major', 'minor', 'patch', 'date'])
    )
    .reset_index(drop=True)
    
    # Uncomment next line to select those that do not match our regex:
    # [lambda d: ~d.release.str.match(r'^(?:(?:v|V)\.?)?(?P<major>\d+)(?:\.(?P<minor>\d+)(?:\.(?P<patch>\d+))?)?(?P<misc>.*)$')]
    # Comment next line to get the extracted major, minor, patch and misc components:
    .drop(columns=['major', 'minor', 'patch', 'misc'])
)

## Helpers

In [11]:
def detect_releases(anchor, releases):
    """
    Given the anchor used in a step to refer to an Action, and a list of (version, date) 
    corresponding to the available releases of an Action in *chronological order*, return a 3-uple of pairs with:
     - selected release, selected release date;
     - first missed release, first missed release date; 
     - last missed release, last missed release date.
    """
    selected = selected_date = None
    first_missed = first_missed_date = None
    last_missed = last_missed_date = None
    
    for version, date in reversed(releases):
        if not selected and version.startswith(anchor):
            selected = version
            selected_date = date
            # no need to go further "in the past"
            break
        else:
            if not last_missed:
                last_missed = version
                last_missed_date = date
            first_missed = version
            first_missed_date = date
            
    return ((selected, selected_date), (first_missed, first_missed_date), (last_missed, last_missed_date))

Some "optimizations": 

 - We group steps by the actions they use, so we only have one lookup to find the corresponding releases;
 - We pass over snapshots in decreasing order, so we can remove "newer" releases as soon as they are not available for the corresponding snapshot; 
 - We only apply the process for steps using an Action for which we have releases;
 - We'll provide a sorted list of releases and dates, at once, so we don't have to sort them each time (see second point above);
 - We group steps that use a same anchor at the same time, to avoid duplicating some computations;

Main steps of the "algorithm": 

    for action, steps in steps.grouped:
        get all releases, sorted by date
        if empty:
            continue with next action

        for snapshot in snapshots_to_consider_sorted_by_date_reversed:
            get available releases

            for distinct anchor, steps in steps using that action:
                S, F, L = detect_releases(anchor, releases available)
                if none selected: continue with next anchor
                for each step in steps:
                    store

In [12]:
output = []

for action, steps in tqdm(df_steps.groupby('action', sort=False), desc='actions', position=0):
    releases = df_releases[lambda d: d.action == action]
    
    if len(releases) == 0:
        continue
        
    for snapshot, snapshot_steps in reversed(list(steps.groupby('snapshot', sort=True))):
        releases = releases[lambda d: d.date <= snapshot]
        
        for anchor, anchor_steps in snapshot_steps.groupby('anchor', sort=False):
            (sel, sel_date), (first, first_date), (last, last_date) = detect_releases(anchor, releases[['release', 'date']].values)
            
            if sel is None:
                continue
            
            for e in anchor_steps.itertuples():
                output.append((
                    e.repository, 
                    snapshot, 
                    e.workflow, 
                    e.job, 
                    e.pos,
                    action,
                    anchor,
                    sel, 
                    sel_date,
                    first,
                    first_date,
                    last,
                    last_date,
                ))      

actions: 100%|██████████████████████████████| 3519/3519 [01:42<00:00, 34.22it/s]


In [13]:
df_lag = (
    pd.DataFrame(data=output, columns=['repository', 'snapshot', 'workflow', 'job', 'step', 'action', 'anchor', 
                'selected', 'selected_date', 'first_missed', 'first_missed_date', 'last_missed', 'last_missed_date'])
    .replace({None: np.nan})
)

In [14]:
df_lag.sample(n=20, random_state=12345)

,repository,snapshot,workflow,job,step,action,anchor,selected,selected_date,first_missed,first_missed_date,last_missed,last_missed_date
979230,joseexposito/touchegg,2021-05-01,ef7d3ce98397555dbe4685105baa7c708589da2331dd43...,rpm-package,3,actions/checkout,v2,v2.3.4,2020-11-03 14:48:49,NaN,NaT,NaN,NaT
83065,pinojs/sonic-boom,2022-08-01,994e14a82f526ca27d831d97549ad9a218cca48882c55d...,test,1,actions/checkout,v3,v3.0.2,2022-04-21 14:56:58,NaN,NaT,NaN,NaT
417221,ben-manes/caffeine,2022-03-01,3c21005f3afc5e609dfe35f4325165633efa6769533d7f...,lincheck,1,actions/checkout,v2.4.0,v2.4.0,2021-11-02 17:48:06,NaN,NaT,NaN,NaT
2260771,lest/prometheus-rpm,2022-09-01,9373da1350f6d57368448c5d2de490a01ff8fff3495c14...,build_packages,2,actions/setup-python,v2,v2.3.2,2022-02-04 11:10:37,v3.0.0,2022-02-28 10:30:59,v4.2.0,2022-08-02 12:44:58
203529,gimli-rs/addr2line,2022-07-01,4058a78ee6e31c8639e673133ab222e0c0534e2eb26f5e...,msrv,1,actions/checkout,v2,v2.4.2,2022-04-21 16:04:02,v3.0.0,2022-03-01 17:48:27,v3.0.2,2022-04-21 14:56:58
321330,supermap/iclient-javascript,2022-05-01,c4dec0f9e1110ab07a1663c1de9bfdc087efb6a433ff15...,mirror_to_gitee,1,actions/checkout,v2,v2.4.2,2022-04-21 16:04:02,v3.0.0,2022-03-01 17:48:27,v3.0.2,2022-04-21 14:56:58
2754518,rodrigo-brito/gocity,2021-01-01,c5b923025e7018078ea72e453cf54d84e39cb0f008fe8a...,goreleaser,2,actions/setup-go,v2,v2.1.3,2020-10-01 14:52:31,NaN,NaT,NaN,NaT
873154,4seer/openflutterecommerceapp,2021-08-01,58baddf90f448d841e6fac013687c13dc047d69b229031...,test,1,actions/checkout,v1,v1.0.0,2019-08-01 12:59:53,v2-beta,2019-12-03 15:35:24,v2.3.4,2020-11-03 14:48:49
2592244,raftario/filite,2019-11-01,9401173a2d60a101bbe20070f651b658be12ebb31d428a...,build,2,actions-rs/toolchain,v1,v1.0.3,2019-10-19 10:31:19,NaN,NaT,NaN,NaT
1118301,pytest-dev/pytest-timeout,2021-02-01,7fb7009427963a7bb34ba70fe7096def0cc17aa3756a12...,build,1,actions/checkout,v1,v1.0.0,2019-08-01 12:59:53,v2-beta,2019-12-03 15:35:24,v2.3.4,2020-11-03 14:48:49


In [15]:
df_lag.to_csv('../data/lag.csv.xz', compression='xz', index=False)